In [57]:
import sys
sys.path.append("../")

import sympy as sp
import pathlib as pl
from SymEigen import *
from sympy import symbols
from project_dir import backend_source_dir
from affine_body_core import compute_J_point, compute_J_vec

Gen = EigenFunctionGenerator()
Gen.MacroBeforeFunction("__host__ __device__")
Gen.DisableLatexComment()

# Articulation

All stuff we need in optimization: 

$$
K = \frac{1}{2}(\delta\theta_i - \tilde{\delta\theta}_i) M_{ij} (\delta\theta_j - \tilde{\delta\theta}_j),
$$

$$
G^{\theta}_i = \frac{\partial K}{\partial \delta\theta_i} = M_{ij} (\delta\theta_j - \tilde{\delta\theta}_j),
$$

$$
H^{\theta}_{ij} = \frac{\partial^2 K}{\partial \delta\theta_i \partial \delta\theta_j} = M_{ij},
$$

$$
\begin{aligned}
G^{F}_i 
&= \frac{\partial K}{\partial \mathbf{F}_i} \\
&= \frac{\partial K}{\partial \delta\theta_i} \frac{\partial \delta\theta_i}{\partial \mathbf{F}_i} \quad (\text{no-ein}) \\
&= G^{\theta}_i \frac{\partial \delta\theta_i}{\partial \mathbf{F}_i} \quad (\text{no-ein}) \\
\end{aligned}
$$

$$
\begin{aligned}
H^{F}_{ij} 
&= \frac{\partial^2 K}{\partial \mathbf{F}_i \partial \mathbf{F}_j}\\
&= \frac{\partial \theta_i}{\partial \mathbf{F}_i} \frac{\partial^2 K}{\partial \theta_i \partial \theta_j} \frac{\partial \theta_j}{\partial \mathbf{F}_j} + \delta_{ij} \frac{\partial K}{\partial \theta_i} \frac{\partial^2 \theta_i}{\partial \mathbf{F}_i \partial \mathbf{F}_i} \quad (\text{no-ein}) \\
&= \frac{\partial \theta_i}{\partial \mathbf{F}_i} M_{ij} \frac{\partial \theta_j}{\partial \mathbf{F}_j} + \delta_{ij} G^{\theta}_i \frac{\partial^2 \theta_i}{\partial \mathbf{F}_i \partial \mathbf{F}_i} \quad (\text{no-ein}) \\
\end{aligned}
$$

For symbol calculation:

$$
\frac{\partial \delta\theta}{\partial \mathbf{F}}
$$

$$
\frac{\partial^2 \delta\theta}{\partial \mathbf{F} \partial \mathbf{F}}
$$

# Mapping to Affine Body Space

$$
G^{q} = J^T G^{F}
$$

$$
H^{q} = J^T H^{F} J
$$

In [58]:
def abd_space_G(G_F, J):
    return J.T * G_F

def abd_space_H(H_F, J):
    return J.T * H_F * J

# Revolute Joint Constraint

![Revolute Joint Constraint](../../docs/specification/constitutions/media/external_articulation_revolute_constraint_fig1.drawio.svg)

We define the joint frame $\mathbf{F}$

$$
\mathbf{F} =
\begin{bmatrix}
\hat{\mathbf{b}}_i \\
\hat{\mathbf{n}}_i \\
\hat{\mathbf{b}}_j \\
\hat{\mathbf{n}}_j
\end{bmatrix}
$$

In [59]:
F = Eigen.Vector("F", 12)
bi = F[0:3, 0]
ni = F[3:6, 0]
bj = F[6:9, 0]
nj = F[9:12, 0]

F_t = Eigen.Vector("F_t", 12)
bi_t = F_t[0:3, 0]
ni_t = F_t[3:6, 0]
bj_t = F_t[6:9, 0]
nj_t = F_t[9:12, 0]

Cl = Gen.Closure(F, F_t)

# display(bi, ni, bj, nj)

From affine body $i$ and $j$ to $\mathbf{F}$:

$$
\begin{aligned}
J^F 
&= \frac{\partial \mathbf{F}}{\partial \mathbf{q}} \\
&=
\begin{bmatrix}
\mathring{\mathbf{J}}(\bar{\mathbf{b}}_i) &  \\
\mathring{\mathbf{J}}(\bar{\mathbf{n}}_i) &  \\
& \mathring{\mathbf{J}}(\bar{\mathbf{b}}_j)  \\
& \mathring{\mathbf{J}}(\bar{\mathbf{n}}_j)  \\
\end{bmatrix}_{12 \times 24} 
\end{aligned}
$$

In [60]:
J_F = sp.Matrix.zeros(12, 24)
B = Eigen.Vector("B", 12)

bi_bar = B[0:3, 0]
ni_bar = B[3:6, 0]
bj_bar = B[6:9, 0]
nj_bar = B[9:12, 0]

J_F[0:3, 0:12] = compute_J_vec(bi_bar)
J_F[3:6, 0:12] = compute_J_vec(ni_bar)
J_F[6:9, 12:24] = compute_J_vec(bj_bar)
J_F[9:12, 12:24] = compute_J_vec(nj_bar)

# display(J_F)

G_F = Eigen.Vector("G_F", 12)
H_F = Eigen.Matrix("H_F", 12, 12)
JT_G = abd_space_G(G_F, J_F)
JT_H_J = abd_space_H(H_F, J_F)

Cl_G_Mapping = Gen.Closure(G_F, B)
Cl_H_Mapping = Gen.Closure(H_F, B)

$$
\cos\theta = \frac{
    \hat{\mathbf{b}}_i \cdot \hat{\mathbf{b}}_j + \hat{\mathbf{n}}_i \cdot \hat{\mathbf{n}}_j
    }{2},
$$

$$
\sin\theta = \frac{
    \hat{\mathbf{n}}_i \cdot \hat{\mathbf{b}}_j - \hat{\mathbf{b}}_i \cdot \hat{\mathbf{n}}_j
    }{2}.
$$

$$
\delta\theta = \arctan \left(\tan \left(\theta - \theta^t \right)\right) = \arctan \frac{\sin\theta \cos\theta^t - \cos\theta \sin\theta^t}{\cos\theta \cos\theta^t + \sin\theta \sin\theta^t},
$$

In [61]:
def compute_cos_theta(bi, ni, bj, nj):
    return (bi.dot(bj) + ni.dot(nj)) / 2

def compute_sin_theta(bi, ni, bj, nj):
    return (ni.dot(bj) - bi.dot(nj)) / 2

cos_theta = compute_cos_theta(bi, ni, bj, nj)
sin_theta = compute_sin_theta(bi, ni, bj, nj)
cos_theta_t = compute_cos_theta(bi_t, ni_t, bj_t, nj_t)
sin_theta_t = compute_sin_theta(bi_t, ni_t, bj_t, nj_t)

In [62]:
tan_delta_theta = (sin_theta * cos_theta_t - cos_theta * sin_theta_t) / (cos_theta * cos_theta_t + sin_theta * sin_theta_t)
delta_theta = sp.atan(tan_delta_theta)

In [63]:
d_delta_theta_dF = VecDiff(delta_theta, F)
dd_delta_theta_ddF = VecDiff(d_delta_theta_dF, F)

In [64]:
s = f"""// Affine Body Revolute Joint Constraint
// Description: Symbolic expressions for revolute joint constraint between two affine bodies.

// F = [bi; ni; bj; nj]
// bi, ni: body i joint frame vectors

// B = [ci_bar; ti_bar; cj_bar; tj_bar]
// Ga = J.T * G_F
// Ha = J.T * H_F * J

{Cl_G_Mapping("JT_G", JT_G)}
{Cl_H_Mapping("JT_H_J", JT_H_J)}
{Cl("dDeltaTheta_dF",d_delta_theta_dF)}
{Cl("ddDeltaTheta_ddF",dd_delta_theta_ddF)}
"""

path = backend_source_dir("cuda") / "affine_body/constraints/sym/external_articulation_revolute_joint_constraint.inl"
print("Written to:", path)
f = open(path, "w")
f.write(s)
f.close()

Written to: /Users/luxinyu/Projects/LibuipcProject/libuipc/src/backends/cuda/affine_body/constraints/sym/external_articulation_revolute_joint_constraint.inl


# Prismatic Joint Constraint

![Prismatic Joint Constraint](../../docs/specification/constitutions/media/external_articulation_prismatic_constraint_fig1.drawio.svg)

$$
    \theta = \frac{
        (\mathbf{c}_j - \mathbf{c}_i) \cdot \hat{\mathbf{t}}_i -
        (\mathbf{c}_i - \mathbf{c}_j) \cdot \hat{\mathbf{t}}_j
    }{2},
$$

$$
    \theta = \frac{
        \mathbf{d}_{ij} \cdot \hat{\mathbf{t}}_i +
        \mathbf{d}_{ij} \cdot \hat{\mathbf{t}}_j
    }{2},
$$

where,

$$
\mathbf{d}_{ij} = \mathbf{c}_j - \mathbf{c}_i
$$

We define the joint frame $\mathbf{F}$

$$
\mathbf{F} =
\begin{bmatrix}
\mathbf{d}_{ij} \\
\hat{\mathbf{t}}_i \\
\hat{\mathbf{t}}_j
\end{bmatrix}
$$

In [65]:
F = Eigen.Vector("F", 9)
dij = F[0:3, 0]
ti = F[3:6, 0]
tj = F[6:9, 0]

F_t = Eigen.Vector("F_t", 9)
dij_t = F_t[0:3, 0]
ti_t = F_t[3:6, 0]
tj_t = F_t[6:9, 0]

def compute_prismatic_theta(F):
    dij = F[0:3, 0]
    ti = F[3:6, 0]
    tj = F[6:9, 0]
    theta = (dij.dot(ti) + dij.dot(tj))/2
    return theta

theta = compute_prismatic_theta(F)
theta_t = compute_prismatic_theta(F_t)

delta_theta = theta_t - theta

d_delta_theta_dF = VecDiff(delta_theta, F)
dd_delta_theta_ddF = VecDiff(d_delta_theta_dF, F)

Cl = Gen.Closure(F, F_t)

From affine body $i$ and $j$ to $\mathbf{F}$:

$$
\begin{aligned}
J^F 
&= \frac{\partial \mathbf{F}}{\partial \mathbf{q}} \\
&=
\begin{bmatrix}
-\mathbf{J}(\bar{\mathbf{c}}_i) & \mathbf{J}(\bar{\mathbf{c}}_j) \\
\mathring{\mathbf{J}}(\bar{\mathbf{t}}_i) &  \\
& \mathring{\mathbf{J}}(\bar{\mathbf{t}}_j) \\
\end{bmatrix}_{9 \times 24} 
\end{aligned}
$$

In [66]:
J_F = sp.Matrix.zeros(9, 24)
# Basis vector
B = Eigen.Vector("B", 12)

ci_bar = B[0:3, 0]
ti_bar = B[3:6, 0]
cj_bar = B[6:9, 0]
tj_bar = B[9:12, 0]

J_F[0:3, 0:12] = -compute_J_point(ci_bar)
J_F[0:3, 12:24] = compute_J_point(cj_bar)
J_F[3:6, 0:12] = compute_J_vec(ti_bar)
J_F[6:9, 12:24] = compute_J_vec(tj_bar)

# display(J_F)

G_F = Eigen.Vector("G_F", 9)
H_F = Eigen.Matrix("H_F", 9, 9)
JT_G = abd_space_G(G_F, J_F)
JT_H_J = abd_space_H(H_F, J_F)

Cl_G_Mapping = Gen.Closure(G_F, B)
Cl_H_Mapping = Gen.Closure(H_F, B)

In [67]:
s = f"""// Affine Body Prismatic Joint Constraint
// Description: Symbolic expressions for prismatic joint constraint between two affine bodies.

// dij = c_j - c_i
// F = [dij; ti; tj]
// F_t = [dij_t; ti_t; tj_t]
// B = [ci_bar; ti_bar; cj_bar; tj_bar]

{Cl_G_Mapping("JT_G", JT_G)}
{Cl_H_Mapping("JT_H_J", JT_H_J)}
{Cl("dDeltaTheta_dF", d_delta_theta_dF)},
{Cl("ddDeltaTheta_ddF", dd_delta_theta_ddF)}
"""

path = backend_source_dir("cuda") / "affine_body/constraints/sym/external_articulation_prismatic_joint_constraint.inl"
print("Written to:", path)
f = open(path, "w")
f.write(s)
f.close()

Written to: /Users/luxinyu/Projects/LibuipcProject/libuipc/src/backends/cuda/affine_body/constraints/sym/external_articulation_prismatic_joint_constraint.inl
